匯入threading

In [1]:
import threading

獲取正在使用的線程數

In [2]:
def main():
  print (threading.active_count())
if __name__ == '__main__':
  main()


7


查看所有線程資訊

In [3]:
def main():
  print(threading.enumerate())

if __name__ == '__main__':
  main()


[<_MainThread(MainThread, started 4571100672)>, <Thread(IOPub, started daemon 123145398558720)>, <Heartbeat(Heartbeat, started daemon 123145415348224)>, <Thread(Thread-2 (_watch_pipe_fd), started daemon 123145433210880)>, <Thread(Thread-3 (_watch_pipe_fd), started daemon 123145450000384)>, <ControlThread(Control, started daemon 123145466789888)>, <HistorySavingThread(IPythonHistorySavingThread, started 123145483579392)>]


目前正在運行的線程

In [4]:
def main():
  print(threading.current_thread())

if __name__ == '__main__':
  main()


<_MainThread(MainThread, started 4571100672)>


添加線程

In [5]:
def thread_job():
    print('This is a thread of %s' % threading.current_thread())

def main():
    thread = threading.Thread(target=thread_job,)   # 定義線程，target代表這個線程所要完成的工作
    thread.start()  # 讓線程開始

if __name__ == '__main__':
    main()


This is a thread of <Thread(Thread-4 (thread_job), started 123145500905472)>


join使用

In [9]:
import threading
import time
def thread_job():
  print("T1 start\n")
  for i in range(10):
    time.sleep(0.1)   # 每次間隔0.1s
  print("T1 finish\n")

def main():
  added_thread = threading.Thread(target=thread_job, name='T1')
  added_thread.start()
  added_thread.join() #等待join以上的thread先跑完才繼續往下執行
  print("all done\n")


if __name__ == '__main__':
    main()


T1 start

T1 finish

all done



Lock使用：確保不同線程使用同一資料時，不會互相影響。
在執行前加上lock.acquire()上鎖
執行完畢後用lock.release()解鎖

為使用lock情況，會發現兩個job的輸出很亂

In [12]:
import threading

def job1():
    global A
    for i in range(10):
        A+=1
        print('job1',A)

def job2():
    global A
    for i in range(10):
        A+=10
        print('job2',A)

if __name__== '__main__':
    lock=threading.Lock()
    A=0
    t1=threading.Thread(target=job1)
    t2=threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110


In [ ]:
加上Lock

In [13]:
import threading

def job1():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+=1
        print('job1',A)
    lock.release()

def job2():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+=10
        print('job2',A)
    lock.release()

if __name__== '__main__':
    lock=threading.Lock() #lock為threading中的一個物件
    A=0
    t1=threading.Thread(target=job1)
    t2=threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110


Queue:儲存線程中的資料
因為線程內的def沒辦法return值

In [14]:
import threading
import time

from queue import Queue #匯入queue


def job(l, q):
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l) #取代return的功能，將回傳值儲存在queue中


def multithreading():
    q = Queue()
    threads = [] #線程list
    data = [[1, 2, 3], [3, 4, 5], [4, 4, 4], [5, 5, 5]]
    for i in range(4):
        t = threading.Thread(target=job, args=(data[i], q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    results = []
    for _ in range(4):
        results.append(q.get())
    print(results)


if __name__ == '__main__':
    multithreading()


[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


GIL(Global Interpreter Lock): GIL 可以用於確保在 Python 運行時僅運行一個 Thread 來保證多執行緒同時運行，並對同一資源進行讀寫操作的修改時，其執行緒與執行緒間不會發生衝突，和數據修改不會發生錯誤（Thread-safe）。

In [20]:
import threading
from queue import Queue
import copy
import time

def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal: ',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading: ', time.time()-s_t)

1999998000000
normal:  0.06059002876281738
1999998000000
multithreading:  0.05639195442199707


thread和Normal做了一樣多次的運算，但可以發現兩者的差距不大，不如預期的3-4倍